In [31]:
#import everything upwards from home dir
from __future__ import absolute_import
import psycopg2
from configparser import ConfigParser
import argparse
import six
import sys
import google.ads.google_ads.client
import pandas as pd
import re
import datetime
import numpy as np
from xlrd import XLRDError
from helper_functions import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

def google_ads(client, customer_id, page_size, df_conf_req):
    ga_service = client.get_service('GoogleAdsService', version='v2')
    channel_types = client.get_type('AdvertisingChannelTypeEnum')
    
    # create a list containing dimensions
    dim_str = ''
    dim_lst = []
    period = df_conf_req.iat[0,1]
    for index, row in df_conf_req.iterrows():
        dim_lst.append(row['dimensions'])
        if row['dimensions'] != 'order_no':
            dim_str = dim_str + row['dimensions'] + ', '
    dim_str = dim_str[:-2]
    
    print('Calling Google Ads API...')
    # make the call to Google Ads using compiled parameters 
    query = f"SELECT {dim_str} FROM campaign WHERE segments.date {period}"

    response = ga_service.search(customer_id, query, page_size=page_size)

    df_response = pd.DataFrame()
    row_count = 0
    
    # iterate over response and add everything to a pandas dataframe
    try:
        for index, row in enumerate(response):
            row_count = index
            new_dim_lst = []
            new_dim_dtype_lst = []
            
            
            # create a list for dimension data types
            for dim in dim_lst:
                if 'order_no' == dim:
                    new_dim_dtype_lst.append('string')
                elif 'int' in str(type(eval('row.' + dim))).lower():
                    new_dim_dtype_lst.append('int')
                elif 'double' in str(type(eval('row.' + dim))).lower():
                    new_dim_dtype_lst.append('float')
                else:
                    new_dim_dtype_lst.append('string')
                
                if 'order_no' == dim:
                    new_dim_lst.append(dim)
                elif 'google' in str(type(eval('row.' + dim))):
                    new_dim_lst.append('row.' + dim + '.value')
                else:
                    new_dim_lst.append('row.' + dim)
            
            # iterate over dimensions of a single row
            for idx, new_dim in enumerate(new_dim_lst):
                if dim_lst[idx] == 'order_no': 
                    df_response.loc[index, dim_lst[idx]] = str(re.findall('(PLN?[\-]\d{1,4}?[\-]\d{1,4})', row.campaign.name.value))[2:-2]
                    df_response[dim_lst[idx]] = df_response[dim_lst[idx]].astype('object')
                elif dim_lst[idx] == 'campaign.advertising_channel_type':
                    df_response.loc[index, dim_lst[idx]] = str(channel_types.AdvertisingChannelType.Name(eval('row.' + dim_lst[idx])))
                    df_response[dim_lst[idx]] = df_response[dim_lst[idx]].astype('object')
                elif dim_lst[idx] == 'metrics.cost_micros':                 
                    df_response.loc[index, dim_lst[idx]] = float(float(eval(new_dim)) / 100000)
                    df_response[dim_lst[idx]] = df_response[dim_lst[idx]].astype('float64')
                elif new_dim_dtype_lst[idx] == 'int':
                    df_response.loc[index, dim_lst[idx]] = int(eval(new_dim))
                    df_response[dim_lst[idx]] = df_response[dim_lst[idx]].astype('int64')
                elif new_dim_dtype_lst[idx] == 'float':
                    df_response.loc[index, dim_lst[idx]] = float(eval(new_dim))
                    df_response[dim_lst[idx]] = df_response[dim_lst[idx]].astype('float64')
                else:
                    df_response.loc[index, dim_lst[idx]] = str(eval(new_dim))
                    df_response[dim_lst[idx]] = df_response[dim_lst[idx]].astype('object')
                
    # return error messages if exception
    except google.ads.google_ads.errors.GoogleAdsException as ex:
        print('Request with ID "%s" failed with status "%s" and includes the '
              'following errors:' % (ex.request_id, ex.error.code().name))
        for error in ex.failure.errors:
            print('\tError with message "%s".' % error.message)
            if error.location:
                for field_path_element in error.location.field_path_elements:
                    print('\t\tOn field: %s' % field_path_element.field_name)
        sys.exit(1)
    print(str(row_count + 1) + ' row(s) received')
    return df_response, dim_lst

def db_config(filename='database.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)
 
    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename)) 
    return db

def postgre_write(df_response, dim_lst):
    # Connect to the PostgreSQL database server
    conn = None
    # read connection parameters
    params = db_config()
    
    # connect to the PostgreSQL server
    print('Connecting to the PostgreSQL database...')
    conn = psycopg2.connect(**params)
 
    try:
        # create a cursor
        cur = conn.cursor()

        # execute initial db write statements 
        # *tread lightly* cur.execute("DROP TABLE google_ads_temp;")
        cur.execute("CREATE TABLE IF NOT EXISTS google_ads_temp("
                       "creation_ts TIMESTAMP , "
                       "last_updated_ts TIMESTAMP, "
                       "campaign_id BIGINT, "
                       "segments_date DATE, "
                       "CONSTRAINT table_pk PRIMARY KEY (campaign_id, segments_date));")
        conn.commit()
        row_count_db = 0
        # iterate over response dataframe and insert to DB
        print('Working...')
        for index, row in df_response.iterrows():
            row_count_db = index
            creation_ts = datetime.datetime.now()
            last_updated_ts = datetime.datetime.now()
            ins_query_dim ='creation_ts, last_updated_ts, '
            ins_query_val = f"'{creation_ts}', '{last_updated_ts}', "
            upd_query = f"last_updated_ts = '{last_updated_ts}', "
            col_dtype = ''
            for idx, dim in enumerate(dim_lst):
                db_dim = dim
                # conditionals for variable type assignment
                if '.' in dim:
                    db_dim = dim.replace('.', '_')
                if str(dim) in ('campaign.id'):
                    col_dtype = 'bigint NOT NULL'
                    cur.execute(f"ALTER TABLE google_ads_temp "
                            f"ADD COLUMN IF NOT EXISTS {db_dim} {col_dtype};")
                    ins_query_dim = ins_query_dim + db_dim + ','
                    ins_query_val = ins_query_val + str(row[dim]) + ', '
                elif 'date' in str(dim):
                    col_dtype = 'date'
                    cur.execute(f"ALTER TABLE google_ads_temp "
                                f"ADD COLUMN IF NOT EXISTS {db_dim} {col_dtype};")
                    ins_query_dim = ins_query_dim + db_dim + ','
                    ins_query_val = ins_query_val + "'" + str(row[dim]) + "', "
                elif 'float' in str(df_response[dim].dtype) or 'micro' in str(dim):
                    col_dtype = 'real'
                    cur.execute(f"ALTER TABLE google_ads_temp "
                                f"ADD COLUMN IF NOT EXISTS {db_dim} {col_dtype};")
                    ins_query_dim = ins_query_dim + db_dim + ','
                    ins_query_val = ins_query_val + " " + str(row[dim]) + ", "
                    upd_query = upd_query + db_dim + " = " + str(row[dim]) + ", "
                elif 'int' in str(df_response[dim].dtype):
                    col_dtype = 'bigint'
                    cur.execute(f"ALTER TABLE google_ads_temp "
                                f"ADD COLUMN IF NOT EXISTS {db_dim} {col_dtype};")
                    ins_query_dim = ins_query_dim + db_dim + ','
                    ins_query_val = ins_query_val + " " + str(row[dim]) + ", "
                    upd_query = upd_query + db_dim + " = " + str(row[dim]) + ", "
                else:
                    col_dtype = 'varchar (150)'
                    cur.execute(f"ALTER TABLE google_ads_temp "
                                f"ADD COLUMN IF NOT EXISTS {db_dim} {col_dtype};")
                    ins_query_dim = ins_query_dim + db_dim + ','
                    ins_query_val = ins_query_val + "'" + str(row[dim]) + "', "
                if str(db_dim) in ('campaign_end_date', 'campaign_name', 'campaign_advertising_channel_type'):
                    upd_query = upd_query + db_dim + " = '" + str(row[dim]) + "', "
            ins_query_dim = ins_query_dim[:-1]
            ins_query_val = ins_query_val[:-2]
            upd_query = upd_query[:-2]

            # insert into db or if row exists update conversions
            cur.execute(f"INSERT INTO google_ads_temp ({ins_query_dim}) "
                        f"VALUES ({ins_query_val}) "
                        f"ON CONFLICT (campaign_id, segments_date) " 
                            f"DO "
                                f"UPDATE "
                                f"SET {upd_query}; ")
            conn.commit()
        print(str(row_count_db + 1) + ' row(s) inserted')
       # close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print('Database error')
        print(error)
        sys.exit(1)
    finally:
        if conn is not None:
            conn.close()
    print('Database connection closed.')
    print('')
                     
def start():
    # If the google-ads.yaml file is present in home dir, GoogleAdsClient will read the configuration.  
    _DEFAULT_PAGE_SIZE = 500
    print('Starting...')
    try:
        # read configuration from excel
        google_ads_client = (google.ads.google_ads.client.GoogleAdsClient.load_from_storage())
        df_conf_base = pd.read_excel('google_ads_conf_1.xlsx', sheet_name='base', header=0)
        df_conf_req = pd.read_excel('google_ads_conf_1.xlsx', sheet_name='parameters', header=0)
        db_config(filename = 'database.ini')
        if pd.isna(df_conf_base['customer_id'].iloc[0]):
            raise KeyError('No base data provided (customer_id(s))')
        if pd.isna(df_conf_req['period'].iloc[0]):
            raise KeyError('Period is missing')    
        for index, row in df_conf_req.iterrows():
            if pd.isna(row['dimensions']):
                raise KeyError('One or more dimensions missing')        
    except(NameError, XLRDError, KeyError) as error:
        print('Error while reading configuration file(s)')
        print(error)
        sys.exit(1)
    
    # iterate over customers
    for index, row in df_conf_base.iterrows():              
        try:
            customer_id = str(int(row['customer_id']))
            #customer_id = str(df_conf_base.iat[0,0])
            print('Customer ID: ' + customer_id)
        except(KeyError) as error:
            print('Could not read column')
            print(error)
            sys.exit(1)
        
        # call defined methods
        google_ads_resp = google_ads(google_ads_client, customer_id, _DEFAULT_PAGE_SIZE, df_conf_req)
        df_response = google_ads_resp[0]
#         dim_lst = google_ads_resp[1]
#         postgre_write(df_response, dim_lst)
#         print('Success')
        print(df_response.head())
    
    
        t_name = 'google_ads_new'
        pk_name = 'google_ads_new_pk'
        pk_lst = ['campaign_id', 'segments_date']
        do_drop = False
        page_size = 10000
        src_col_name = 'campaign_name'
        is_pln_df = True

        postgre_write_main(df_response, t_name, pk_name, pk_lst, do_drop, page_size, src_col_name, is_pln_df)
        print('Success')
        
    return df_response
        
try:
    start()
except(KeyError) as error:
    print('error key')

Starting...
Customer ID: 4353554621
Calling Google Ads API...
21 row(s) received
   campaign.id                                      campaign.name  \
0   1081886360  PLN-18-422_Search - Kapsules - kava [A:All][C:...   
1   1081886360  PLN-18-422_Search - Kapsules - kava [A:All][C:...   
2   1081886360  PLN-18-422_Search - Kapsules - kava [A:All][C:...   
3   1081886360  PLN-18-422_Search - Kapsules - kava [A:All][C:...   
4   1081886360  PLN-18-422_Search - Kapsules - kava [A:All][C:...   

   campaign.status campaign.start_date campaign.end_date segments.date  \
0                2          2018-03-09        2037-12-30    2019-10-01   
1                2          2018-03-09        2037-12-30    2019-10-02   
2                2          2018-03-09        2037-12-30    2019-10-03   
3                2          2018-03-09        2037-12-30    2019-10-04   
4                2          2018-03-09        2037-12-30    2019-10-05   

   metrics.impressions  metrics.clicks  metrics.conversions

UnboundLocalError: local variable 'conn' referenced before assignment

In [9]:
#import everything upwards from home dir
from __future__ import absolute_import
import psycopg2
from configparser import ConfigParser
import argparse
import six
import sys
import google.ads.google_ads.client
import pandas as pd
import re
import datetime
import numpy as np
from xlrd import XLRDError

client = (google.ads.google_ads.client.GoogleAdsClient.load_from_storage())
customer_service = client.get_service('CustomerService', version='v2')

try:
    accessible_customers = customer_service.list_accessible_customers()
    result_total = len(accessible_customers.resource_names)
    print('Total results: %i' % result_total)

    resource_names = accessible_customers.resource_names
    for resource_name in resource_names:
        print('Customer resource name: "%s"' % resource_name)
except google.ads.google_ads.errors.GoogleAdsException as ex:
    print('Request with ID "%s" failed with status "%s" and includes the '
          'following errors:' % (ex.request_id, ex.error.code().name))
    for error in ex.failure.errors:
        print('\tError with message "%s".' % error.message)
        if error.location:
            for field_path_element in error.location.field_path_elements:
                print('\t\tOn field: %s' % field_path_element.field_name)
    sys.exit(1)


Total results: 8
Customer resource name: "customers/9493112034"
Customer resource name: "customers/6155439663"
Customer resource name: "customers/2163066616"
Customer resource name: "customers/8531037207"
Customer resource name: "customers/9077667200"
Customer resource name: "customers/1912380076"
Customer resource name: "customers/7932398349"
Customer resource name: "customers/5470129919"


In [30]:
df_response.head()

NameError: name 'google_ads_resp' is not defined